In [1]:
import time
import math

class TicTacToe:
    def __init__(self):
        self.board = [' ' for _ in range(9)]  #3x3 board
        self.current_winner = None

    def print_board(self):
        for row in [self.board[i*3:(i+1)*3] for i in range(3)]:
            print('| ' + ' | '.join(row) + ' |')

    @staticmethod
    def print_board_nums():
        #0 | 1 | 2 etc (tells us what number corresponds to what box)
        number_board = [[str(i) for i in range(j*3, (j+1)*3)] for j in range(3)]
        for row in number_board:
            print('| ' + ' | '.join(row) + ' |')

    def available_moves(self):
        return [i for i, spot in enumerate(self.board) if spot == ' ']

    def empty_squares(self):
        return ' ' in self.board

    def num_empty_squares(self):
        return self.board.count(' ')

    def make_move(self, square, letter):
        if self.board[square] == ' ':
            self.board[square] = letter
            if self.winner(square, letter):
                self.current_winner = letter
            return True
        return False

    def winner(self, square, letter):
        #check row
        row_ind = square // 3
        row = self.board[row_ind*3 : (row_ind + 1)*3]
        if all([spot == letter for spot in row]):
            return True
        
        #check column
        col_ind = square % 3
        column = [self.board[col_ind+i*3] for i in range(3)]
        if all([spot == letter for spot in column]):
            return True
        
        #check diagonals
        if square % 2 == 0:
            diagonal1 = [self.board[i] for i in [0, 4, 8]]  # top-left to bottom-right
            if all([spot == letter for spot in diagonal1]):
                return True
            diagonal2 = [self.board[i] for i in [2, 4, 6]]  # top-right to bottom-left
            if all([spot == letter for spot in diagonal2]):
                return True
        
        return False

def play(game, x_player, o_player, print_game=True):
    if print_game:
        game.print_board_nums()

    letter = 'X'  #starting letter
    
    while game.empty_squares():
        if letter == 'O':
            square = o_player.get_move(game)
        else:
            square = x_player.get_move(game)

        if game.make_move(square, letter):
            if print_game:
                print(letter + f' makes a move to square {square}')
                game.print_board()
                print('')  #empty line

            if game.current_winner:
                if print_game:
                    print(letter + ' wins!')
                return letter

            letter = 'O' if letter == 'X' else 'X'  #switch player

        time.sleep(0.8)  #tiny break for readability

    if print_game:
        print('It\'s a tie!')

class Player:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        pass

class HumanPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)

    def get_move(self, game):
        valid_square = False
        val = None
        while not valid_square:
            square = input(self.letter + '\'s turn. Input move (0-8): ')
            try:
                val = int(square)
                if val not in game.available_moves():
                    raise ValueError
                valid_square = True
            except ValueError:
                print('Invalid square. Try again.')
        return val

class MinimaxAIPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)
        self.nodes_visited = 0

    def get_move(self, game):
        self.nodes_visited = 0
        if len(game.available_moves()) == 9:
            square = random.choice(game.available_moves())  #choose randomly for first move
        else:
            square = self.minimax(game, self.letter)['position']
        print(f"Minimax visited {self.nodes_visited} nodes")
        return square

    def minimax(self, state, player):
        self.nodes_visited += 1
        max_player = self.letter  
        other_player = 'O' if player == 'X' else 'X'

        #first check if the previous move is a winner
        if state.current_winner == other_player:
            return {'position': None, 'score': 1 * (state.num_empty_squares() + 1) if other_player == max_player else -1 * (
                        state.num_empty_squares() + 1)}
        elif not state.empty_squares():
            return {'position': None, 'score': 0}

        if player == max_player:
            best = {'position': None, 'score': -math.inf}  #each score should maximize
        else:
            best = {'position': None, 'score': math.inf}  #each score should minimize

        for possible_move in state.available_moves():
            state.make_move(possible_move, player)
            sim_score = self.minimax(state, other_player)
            state.board[possible_move] = ' '
            state.current_winner = None
            # step 4: update the dictionaries if necessary
            sim_score['position'] = possible_move

            if player == max_player:  # maximizing player
                if sim_score['score'] > best['score']:
                    best = sim_score
            else:  # minimizing player
                if sim_score['score'] < best['score']:
                    best = sim_score

        return best

class AlphaBetaAIPlayer(Player):
    def __init__(self, letter):
        super().__init__(letter)
        self.nodes_visited = 0

    def get_move(self, game):
        self.nodes_visited = 0
        if len(game.available_moves()) == 9:
            square = random.choice(game.available_moves())  # choose randomly for first move
        else:
            square = self.alphabeta(game, self.letter, -math.inf, math.inf)['position']
        print(f"Alpha-Beta visited {self.nodes_visited} nodes")
        return square

    def alphabeta(self, state, player, alpha, beta):
        self.nodes_visited += 1
        max_player = self.letter  # yourself
        other_player = 'O' if player == 'X' else 'X'

        # first check if the previous move is a winner
        if state.current_winner == other_player:
            return {'position': None, 'score': 1 * (state.num_empty_squares() + 1) if other_player == max_player else -1 * (
                        state.num_empty_squares() + 1)}
        elif not state.empty_squares():
            return {'position': None, 'score': 0}

        if player == max_player:
            best = {'position': None, 'score': -math.inf}  # each score should maximize
        else:
            best = {'position': None, 'score': math.inf}  # each score should minimize

        for possible_move in state.available_moves():
            # step 1: make a move, try that spot
            state.make_move(possible_move, player)
            # step 2: recurse using alphabeta to simulate game after that move
            sim_score = self.alphabeta(state, other_player, alpha, beta)
            # step 3: undo the move
            state.board[possible_move] = ' '
            state.current_winner = None
            # step 4: update the dictionaries if necessary
            sim_score['position'] = possible_move

            if player == max_player:  # maximizing player
                if sim_score['score'] > best['score']:
                    best = sim_score
                alpha = max(alpha, best['score'])
            else:  # minimizing player
                if sim_score['score'] < best['score']:
                    best = sim_score
                beta = min(beta, best['score'])
            
            if beta <= alpha:
                break

        return best

if __name__ == '__main__':
    import random
    
    print("Tic-Tac-Toe with AI Players")
    print("1. Human vs Minimax AI")
    print("2. Human vs Alpha-Beta AI")
    print("3. Minimax AI vs Alpha-Beta AI")
    choice = input("Choose game mode (1-3): ")
    
    x_player = None
    o_player = None
    
    if choice == '1':
        x_player = HumanPlayer('X')
        o_player = MinimaxAIPlayer('O')
    elif choice == '2':
        x_player = HumanPlayer('X')
        o_player = AlphaBetaAIPlayer('O')
    elif choice == '3':
        x_player = MinimaxAIPlayer('X')
        o_player = AlphaBetaAIPlayer('O')
    else:
        print("Invalid choice. Defaulting to Human vs Minimax AI.")
        x_player = HumanPlayer('X')
        o_player = MinimaxAIPlayer('O')
    
    t = TicTacToe()
    play(t, x_player, o_player, print_game=True)

Tic-Tac-Toe with AI Players
1. Human vs Minimax AI
2. Human vs Alpha-Beta AI
3. Minimax AI vs Alpha-Beta AI


Choose game mode (1-3):  1


| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |


X's turn. Input move (0-8):  0


X makes a move to square 0
| X |   |   |
|   |   |   |
|   |   |   |

Minimax visited 59705 nodes
O makes a move to square 4
| X |   |   |
|   | O |   |
|   |   |   |



...

X makes a move to square 6
| X | X | O |
|   | O |   |
| X |   |   |

Minimax visited 47 nodes
O makes a move to square 3
| X | X | O |
| O | O |   |
| X |   |   |



X's turn. Input move (0-8):  5


X makes a move to square 5
| X | X | O |
| O | O | X |
| X |   |   |

Minimax visited 5 nodes
O makes a move to square 7
| X | X | O |
| O | O | X |
| X | O |   |



X's turn. Input move (0-8):  8


X makes a move to square 8
| X | X | O |
| O | O | X |
| X | O | X |

It's a tie!
